In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_csv('all_df.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 72 columns):
Unnamed: 0          int64
px                  float64
pz                  float64
start_speed         float64
end_speed           float64
spin_rate           float64
spin_dir            float64
break_angle         float64
break_length        float64
break_y             float64
ax                  float64
ay                  float64
az                  float64
sz_bot              float64
sz_top              float64
type_confidence     float64
vx0                 float64
vy0                 float64
vz0                 float64
x                   float64
x0                  float64
y                   float64
y0                  float64
z0                  float64
pfx_x               float64
pfx_z               float64
nasty               float64
zone                float64
code                object
type                object
type_of_pitch       object
event_num           int6

In [3]:
### will divide df into different dataframes by season

In [4]:
#convert the game id column to a string
dfy = df.g_id.astype('str')

In [5]:
#Since the year is the first four characters, we will isolate those characters
dfy = dfy.str[:4]

In [6]:
dfy.head()

0    2015
1    2015
2    2015
3    2015
4    2015
Name: g_id, dtype: object

In [7]:
#make new columnfor year
df['year'] = dfy.values

In [20]:
dfid= df.ab_id.astype('str')

In [21]:
dfid=dfid.str[5:]

In [22]:
dfid.head(1)

0    00001
Name: ab_id, dtype: object

In [23]:
df['abid']=dfid.values

In [10]:
df.rename(columns= {'Pitchers Name': 'Pitcher_name', 'Batters Name': 'Batter_name', 'b_score': 'batter_score', 'b_count': 'ball_count', 'p_throws': 'pitcher_throws', 'stand': 'batter_stance', 'p_score': 'pitcher_score', 's_count': 'strike_count', 'pitch_type':'type_of_pitch', 'pitch_num': 'pitch_num_ab'}, inplace= True)

In [24]:
df1=df[['abid', "batter_score", 'pitcher_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab', 'on_1b', 'on_2b', 'on_3b' ]].astype(int)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 10 columns):
abid             int64
batter_score     int64
pitcher_score    int64
ball_count       int64
strike_count     int64
outs             int64
pitch_num_ab     int64
on_1b            int64
on_2b            int64
on_3b            int64
dtypes: int64(10)
memory usage: 218.7 MB


##### will get rid of unneccesary data for a new Dataframe, divide data by seasons since there is almost 3 million data points

In [29]:
main= pd.DataFrame(df[['type_of_pitch', 'abid', 'Pitcher_name', 'Batter_name', 'pitcher_score', 'batter_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab',
                            'on_1b', 'on_2b', 'on_3b', 'batter_id', 'inning', 'pitcher_throws',
                             'pitcher_id', 'batter_stance', 'top', 'weather', 'year']])

In [42]:
main['abid'].astype('int64')
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 21 columns):
type_of_pitch     object
abid              object
Pitcher_name      object
Batter_name       object
inning            int64
pitcher_score     int64
batter_score      int64
ball_count        int64
strike_count      int64
outs              int64
pitch_num_ab      int64
on_1b             int64
on_2b             int64
on_3b             int64
pitcher_throws    object
batter_stance     object
weather           object
pitcher_id        int64
batter_id         int64
top               bool
year              object
dtypes: bool(1), int64(12), object(8)
memory usage: 440.2+ MB


In [43]:
#needed to convert all floats to int, no benefit to keeping as floats
main[["batter_score", 'pitcher_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab', 'on_1b', 'on_2b', 'on_3b' ]]= main[["batter_score", 'pitcher_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab', 'on_1b', 'on_2b', 'on_3b' ]].astype(int)

In [44]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 21 columns):
type_of_pitch     object
abid              object
Pitcher_name      object
Batter_name       object
inning            int64
pitcher_score     int64
batter_score      int64
ball_count        int64
strike_count      int64
outs              int64
pitch_num_ab      int64
on_1b             int64
on_2b             int64
on_3b             int64
pitcher_throws    object
batter_stance     object
weather           object
pitcher_id        int64
batter_id         int64
top               bool
year              object
dtypes: bool(1), int64(12), object(8)
memory usage: 440.2+ MB


In [45]:
main= main[['type_of_pitch', 'abid', 'Pitcher_name', 'Batter_name', 'inning', 'pitcher_score', 'batter_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab',
                            'on_1b', 'on_2b', 'on_3b',  'pitcher_throws', 'batter_stance', 'weather',
                             'pitcher_id', 'batter_id', 'top', 'year']]

In [46]:
main.tail(1)

,type_of_pitch,abid,Pitcher_name,Batter_name,inning,pitcher_score,batter_score,ball_count,strike_count,outs,pitch_num_ab,on_1b,on_2b,on_3b,pitcher_throws,batter_stance,weather,pitcher_id,batter_id,top,year
2867153,FF,85574,Josh Hader,Anthony Rizzo,9,3,1,1,2,2,4,0,1,0,L,L,"65 degrees, overcast",623352,519203,False,2018


In [47]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 21 columns):
type_of_pitch     object
abid              object
Pitcher_name      object
Batter_name       object
inning            int64
pitcher_score     int64
batter_score      int64
ball_count        int64
strike_count      int64
outs              int64
pitch_num_ab      int64
on_1b             int64
on_2b             int64
on_3b             int64
pitcher_throws    object
batter_stance     object
weather           object
pitcher_id        int64
batter_id         int64
top               bool
year              object
dtypes: bool(1), int64(12), object(8)
memory usage: 440.2+ MB


In [48]:
#checking for null values
main.isnull().sum()

type_of_pitch     14189
abid                  0
Pitcher_name          0
Batter_name           0
inning                0
pitcher_score         0
batter_score          0
ball_count            0
strike_count          0
outs                  0
pitch_num_ab          0
on_1b                 0
on_2b                 0
on_3b                 0
pitcher_throws        0
batter_stance         0
weather               0
pitcher_id            0
batter_id             0
top                   0
year                  0
dtype: int64

#### Since <1% of the total observations contain null values (under pitch_type)we can drop the null values from the dataset 

In [49]:
#dropping null values
main = main.dropna()

In [50]:
main.isnull().sum()

type_of_pitch     0
abid              0
Pitcher_name      0
Batter_name       0
inning            0
pitcher_score     0
batter_score      0
ball_count        0
strike_count      0
outs              0
pitch_num_ab      0
on_1b             0
on_2b             0
on_3b             0
pitcher_throws    0
batter_stance     0
weather           0
pitcher_id        0
batter_id         0
top               0
year              0
dtype: int64

In [51]:
main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2852965 entries, 0 to 2867153
Data columns (total 21 columns):
type_of_pitch     object
abid              object
Pitcher_name      object
Batter_name       object
inning            int64
pitcher_score     int64
batter_score      int64
ball_count        int64
strike_count      int64
outs              int64
pitch_num_ab      int64
on_1b             int64
on_2b             int64
on_3b             int64
pitcher_throws    object
batter_stance     object
weather           object
pitcher_id        int64
batter_id         int64
top               bool
year              object
dtypes: bool(1), int64(12), object(8)
memory usage: 459.8+ MB


In [52]:
#exploring types of pitch that can be dropped
main['type_of_pitch'].value_counts()

FF    1014877
SL     450578
FT     337983
CH     292789
SI     242504
CU     234391
FC     149756
KC      66484
FS      43705
KN      11260
IN       6197
EP        815
FO        810
PO        628
SC        113
UN         57
AB          9
FA          9
Name: type_of_pitch, dtype: int64

In [53]:
#dropping rows where type_of_pitch not significant and knuckleballs (Knuckleballer only throw knuckle balls)
#removing pitch types with <100 observations
main = main[main.type_of_pitch != 'AB']
main = main[main.type_of_pitch != 'UN']
main = main[main.type_of_pitch != 'EP']
main = main[main.type_of_pitch != 'SC']
main = main[main.type_of_pitch != 'FA']
main = main[main.type_of_pitch != 'PO']
main = main[main.type_of_pitch != 'FO']
main = main[main.type_of_pitch != 'IN']
main = main[main.type_of_pitch != 'KN']

In [54]:
main['type_of_pitch'].value_counts()

FF    1014877
SL     450578
FT     337983
CH     292789
SI     242504
CU     234391
FC     149756
KC      66484
FS      43705
Name: type_of_pitch, dtype: int64

In [55]:
main.head(1)

,type_of_pitch,abid,Pitcher_name,Batter_name,inning,pitcher_score,batter_score,ball_count,strike_count,outs,pitch_num_ab,on_1b,on_2b,on_3b,pitcher_throws,batter_stance,weather,pitcher_id,batter_id,top,year
0,FF,00001,Jon Lester,Matt Carpenter,1,0,0,0,0,0,1,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015


#### One hot encode Balls and Strikes Maybe

### Create target feature 1= Fastball 2= Offspeed 3= Breaking Ball for Pitch Type

#grouped health scores into 4 groups
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 99:
        health_scores['health_scores'].append('1')
    elif x>=96:
        health_scores['health_scores'].append('2')
    elif x >=92:
        health_scores['health_scores'].append('3')
    else :
        health_scores['health_scores'].append('4')
#df
df['health_scores']= health_scores['health_scores']

Pitch Type Definitions
* FT - Two-seam Fastball- 1
* FC - Cutter- 1
* FF - Four-seam Fastball- 1
* SI - Sinker- 1
* FS - Splitter- 2
* CH - Changeup- 2
* SL - Slider- 3
* KC - Knuckle curve- 3
* CU - Curveball- 3

In [56]:
#grouped 'type of pitch in Pitch_Type' (the specific pitch into their classification) into 3 groups: 1=fastball 2= offspeed 3= Breaking pitch
Pitch_Type= {'Pitch_Type' : []}
for x in main['type_of_pitch']:
    if (x== 'FT') or (x=='FC') or (x=='FF'):
        Pitch_Type['Pitch_Type'].append('1')
    elif (x== 'FS') or (x=='CH'):
        Pitch_Type['Pitch_Type'].append('2')
    else:
        Pitch_Type['Pitch_Type'].append('3')

main['Pitch_Type']= Pitch_Type['Pitch_Type']        


    

In [57]:
#run difference feature
main['run_difference']= main['pitcher_score']- main['batter_score']

In [60]:
main= main[['Pitch_Type','type_of_pitch', 'abid', 'Pitcher_name', 'Batter_name', 'inning', 'run_difference', 'pitcher_score', 'batter_score', 'ball_count', 'strike_count', 'outs', 'pitch_num_ab',
                            'on_1b', 'on_2b', 'on_3b',  'pitcher_throws', 'batter_stance', 'weather',
                             'pitcher_id', 'batter_id', 'top', 'year']]
main.head(5)

,Pitch_Type,type_of_pitch,abid,Pitcher_name,Batter_name,inning,run_difference,pitcher_score,batter_score,ball_count,strike_count,outs,pitch_num_ab,on_1b,on_2b,on_3b,pitcher_throws,batter_stance,weather,pitcher_id,batter_id,top,year
0,1,FF,00001,Jon Lester,Matt Carpenter,1,0,0,0,0,0,0,1,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015
1,1,FF,00001,Jon Lester,Matt Carpenter,1,0,0,0,0,1,0,2,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015
2,1,FF,00001,Jon Lester,Matt Carpenter,1,0,0,0,0,2,0,3,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015
3,1,FF,00001,Jon Lester,Matt Carpenter,1,0,0,0,0,2,0,4,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015
4,3,CU,00001,Jon Lester,Matt Carpenter,1,0,0,0,1,2,0,5,0,0,0,L,L,"44 degrees, clear",452657,572761,True,2015


In [ ]:
### Divide seasons convert to Csv for explorationa and modeling

In [61]:
#isolating the data that only contains the 2015 season
df15 = main[main['year'] == '2015']
df15.to_csv('df15.csv')

In [62]:
#isolating the data that only contains the 2016 season
df16 = main[main['year'] == '2016']
df16.to_csv('df16.csv')

In [63]:
#isolating the data that only contains the 2017 season
df17 = main[main['year'] == '2017']
df17.to_csv('df17.csv')

In [64]:
#isolating the data that only contains the 2018 season
df18 = main[main['year'] == '2018']
df18.to_csv('df18.csv')